# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

### Requirements
- pip install SEDkit
- pip install bokeh

In [25]:
# Imports
from locals import source, catalog
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import astropy.table as at
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Making an SED for a single source (no WFSS spectra)

For each spectrum in our WFSS data products, a `Source` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types. Then it assembles an SED from the available data and calculates fundamental and atmospheric parameters.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). Here is what `locals` is doing for each source under the hood.

In [26]:
# Coordinates of LHS 2924 (Usually taken from the WFSS source_list)
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the source object
src = source.Source(ra=ra, dec=dec, name='LHS 2924', spectral_type='L3.5')

# Find some photometry
src.find_2MASS()
src.find_WISE()

# Find a distance from Gaia DR2
src.find_Gaia()

# Take a look
print('\n',src.photometry,'\n')

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.10127179992838364 solRad>, <Quantity 0.02008219196144978 solRad>)
1 2MASS records found.
1 WISE records found.

   band          eff         app_magnitude ...      abs_flux_unc      bandpass
                 um                       ... erg / (Angstrom cm2 s)         
-------- ------------------ ------------- ... ---------------------- --------
  Gaia.G 0.6597013110557894       16.6299 ...                    0.0   Gaia.G
 2MASS.J 1.2393093155660664 11.9899997711 ... 2.6782632688510406e-17  2MASS.J
 2MASS.H 1.6494947091246095 11.2250003815 ... 2.3139101841398836e-17  2MASS.H
2MASS.Ks  2.163860596453316 10.7440004349 ... 1.3684444634813076e-17 2MASS.Ks
 WISE.W1 3.3897048577485647 10.4280004501 ...  3.543992004006416e-18  WISE.W1
 WISE.W2    4.6406356615375 10.1619997025 ... 1.3561312900080802e-18  WISE.W2
 WISE.W3 12.567593607521216 9.64599990845 ...  7.026240830733788e-20  WISE.W3
 WISE.W4 22.31422559

Now we can calculate the fundamental parameters and plot the results.

In [27]:
src.results

param,value,unc,units
object,object,object,object
name,LHS 2924,--,--
age,6.0,4.0,Gyr
distance,10.99,0.02,pc
parallax,90.9962,0.12710000574588776,mas
radius,0.10127179992838364,0.02008219196144978,solRad
spectral_type,73.5,0.5,--
membership,--,--,--
fbol,6.95e-11,7.89e-13,erg / (cm2 s)
mbol,13.913,0.012,--


In [28]:
fig = src.plot()

## Make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output. The difference here is that a WFSS spectrum will be included in each `Source`.

In [29]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/wfss/')

# Initialize empty source catalog
cat = catalog.SourceCatalog(cat_path)

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
1 2MASS records found.
1 WISE records found.
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
1 2MASS records found.
1 WISE records found.


Let's see the source list of all objects in this catalog.

In [30]:
cat.source_list

id,xcentroid,ycentroid,ra_icrs_centroid,dec_icrs_centroid,xmin,xmax,ymin,ymax,abmag,abmag_error,sky_bbox_ll,sky_bbox_ur,sky_bbox_lr,sky_bbox_ul,icrs_centroid
,pix,pix,,,pix,pix,pix,pix,,,"deg,deg","deg,deg","deg,deg","deg,deg","deg,deg"
int64,float64,float64,object,object,float64,float64,float64,float64,int64,float64,object,object,object,object,object
10,1023.89938168,1023.19367348,-3.4283850489866588e-06,-2.694720849093827e-06,940.0,1107.0,943.0,1106.0,9,0.0816191841269,"0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","89.7455,-29.05744"
13,1023.89938168,1023.19367348,-3.4283850489866588e-06,-2.694720849093827e-06,940.0,1107.0,943.0,1106.0,9,0.0816191841269,"0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","217.180137,33.177539"


# TODO: color cuts

All the sources are stored as `Source` instances within the `SourceCatalog.sources` attribute. 

For example, LHS 2924 is part of this catalog so let's make sure we get the same results as when it is run individually.

In [32]:
# Get the source instance
lhs2924 = cat.sources[1]

# Calculate the results
print(lhs2924.results)

# Plot it
fig = lhs2924.plot()

    param           value                unc             units    
------------- ------------------ ------------------- -------------
         name          Source 13                  --            --
          age                6.0                 4.0           Gyr
     distance              10.99                0.02            pc
     parallax            90.9962 0.12710000574588776           mas
       radius              0.101                0.02        solRad
spectral_type                 --                  --            --
   membership                 --                  --            --
         fbol           6.82e-11            4.53e-13 erg / (cm2 s)
         mbol             13.933               0.007            --
         Lbol           9.86e+29            7.47e+27       erg / s
     Lbol_sun              -3.59               0.003            --
         Mbol             13.728               0.007            --
         logg                4.5                0.07          

Hooray!

Let's take a look at the other one, 2MASS 0558-2903, an extreme subdwarf M7.

In [33]:
# Get the source instance
subdwarf = cat.sources[0]
subdwarf.radius = 0.11*q.R_sun, 0.01*q.R_sun

# Calculate the results
print(subdwarf.results)

# Plot it
fig = subdwarf.plot()

Setting radius to (<Quantity 0.11 solRad>, <Quantity 0.01 solRad>)


TypeError: 'NoneType' object is not subscriptable

Fantastic!

Let's see a table of the results.

In [20]:
cat.results

name,age [Gyr],distance [pc],parallax [mas],radius [solRad],spectral_type [--],membership [--],fbol [erg / (cm2 s)],mbol [--],Lbol [erg / s],Lbol_sun [--],Mbol [--],logg [--],mass [solMass],Teff [K]
str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31
Source 10,6.0 +/- 4.0,79.19 +/- 0.68,12.6276 +/- 0.10779999941587448,0.11 +/- 0.01,--,--,3.48e-12 +/- 1.89e-14,17.164 +/- 0.006,2.61e+30 +/- 4.7e+28,-3.17 +/- 0.008,12.671 +/- 0.011,4.5 +/- 0.07,0.8896913720017506 +/- 0.0,2812 +/- 128
Source 13,6.0 +/- 4.0,10.99 +/- 0.02,90.9962 +/- 0.12710000574588776,0.101 +/- 0.02,73.5 +/- 0.5,--,6.82e-11 +/- 4.53e-13,13.933 +/- 0.007,9.86e+29 +/- 7.47e+27,-3.59 +/- 0.003,13.728 +/- 0.007,4.5 +/- 0.07,0.8896913720017506 +/- 0.0,2300 +/- 227
